# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/sehaj/Downloads/Starter_Code 20/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hasaki,35.7333,140.8333,22.38,71,100,1.87,JP,1696470831
1,1,kapuskasing,49.4169,-82.4331,21.00,68,100,6.17,CA,1696470662
2,2,albany,42.6001,-73.9662,21.23,88,0,1.56,US,1696470710
3,3,dunedin,-45.8742,170.5036,21.71,66,100,2.68,NZ,1696470610
4,4,talnakh,69.4865,88.3972,-3.45,95,100,3.22,RU,1696470831


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = .7,
    color = "City",
    xlabel="Longitude",
    ylabel="Latitude",
    title= "Cities by Humidity"
)

city_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27
min_temp = 21
max_wind = 4.5
max_cloud = 0

city_data_df_filter = city_data_df[(city_data_df['Max Temp'] < max_temp) &
                                   (city_data_df['Max Temp'] > min_temp) &
                                   (city_data_df['Wind Speed'] < max_wind) &
                                   (city_data_df['Cloudiness'] == max_cloud)]

# Drop any rows with null values
city_data_df_filter = city_data_df_filter.dropna()

# Display sample data
city_data_df_filter

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,albany,42.6001,-73.9662,21.23,88,0,1.56,US,1696470710
89,89,antonio enes,-16.2325,39.9086,22.79,89,0,1.42,MZ,1696470850
108,108,kill devil hills,36.0307,-75.6760,22.21,87,0,4.12,US,1696470855
124,124,jamestown,42.0970,-79.2353,22.06,72,0,2.57,US,1696470702
149,149,roma,43.2128,-75.4557,22.15,83,0,2.06,US,1696470865
151,151,morondava,-20.2833,44.2833,22.76,75,0,1.25,MG,1696470865
154,154,ubari,26.5921,12.7805,21.76,39,0,4.03,LY,1696470866
181,181,emerald,-23.5333,148.1667,25.75,50,0,2.47,AU,1696470872
223,223,callaway,30.1530,-85.5699,24.44,85,0,0.89,US,1696470881
225,225,pacific grove,36.6177,-121.9166,23.53,67,0,2.57,US,1696470761


### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filter[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,88,
89,antonio enes,MZ,-16.2325,39.9086,89,
108,kill devil hills,US,36.0307,-75.6760,87,
124,jamestown,US,42.0970,-79.2353,72,
149,roma,US,43.2128,-75.4557,83,
151,morondava,MG,-20.2833,44.2833,75,
154,ubari,LY,26.5921,12.7805,39,
181,emerald,AU,-23.5333,148.1667,50,
223,callaway,US,30.1530,-85.5699,85,
225,pacific grove,US,36.6177,-121.9166,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [5]:
# Set parameters to search for a hotel
radius = 10000
limit = 10
params = {
    "limit":limit,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
antonio enes - nearest hotel: Hotel Quirimbas
kill devil hills - nearest hotel: Mariner Days Inn & Suites
jamestown - nearest hotel: DoubleTree Jamestown
roma - nearest hotel: Wingate by Wyndham Rome
morondava - nearest hotel: Hotel Sharon Menabe
ubari - nearest hotel: فندق اوباري
emerald - nearest hotel: Emerald Central Hotel
callaway - nearest hotel: Holiday Inn Express & Suites Panama City-Tyndall
pacific grove - nearest hotel: Pacific Grove Inn
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
toliara - nearest hotel: Ambary
page - nearest hotel: Clarion Inn Page - Lake Powell
beaconsfield - nearest hotel: Chez Moi
ocean city - nearest hotel: Watson's Regency Suites
hamilton - nearest hotel: North Vista Manor
hammonton - nearest hotel: No hotel found
brigantine - nearest hotel: La Sammana
ad dawadimi - nearest hotel: No hotel found
panama city beach - nearest hotel: Shores of Panama
ravar - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,88,No hotel found
89,antonio enes,MZ,-16.2325,39.9086,89,Hotel Quirimbas
108,kill devil hills,US,36.0307,-75.6760,87,Mariner Days Inn & Suites
124,jamestown,US,42.0970,-79.2353,72,DoubleTree Jamestown
149,roma,US,43.2128,-75.4557,83,Wingate by Wyndham Rome
151,morondava,MG,-20.2833,44.2833,75,Hotel Sharon Menabe
154,ubari,LY,26.5921,12.7805,39,فندق اوباري
181,emerald,AU,-23.5333,148.1667,50,Emerald Central Hotel
223,callaway,US,30.1530,-85.5699,85,Holiday Inn Express & Suites Panama City-Tyndall
225,pacific grove,US,36.6177,-121.9166,67,Pacific Grove Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

vaca_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    xlabel="Longitude",
    ylabel="Latitude",
    title= "Cities by Humidity",
    hover_cols = ["Hotel Name", "Country"]
)

vaca_plot